# Load Config

In [1]:
import os
from hydra import initialize_config_dir, compose

os.chdir('..')
from marsbench.utils.config_mapper import load_dynamic_configs
config_dir = os.path.abspath('marsbench/configs')
with initialize_config_dir(config_dir=config_dir, version_base="1.1"):
    cfg = compose(
        config_name="config",
        overrides=[
            "task=classification",
            "model_name=vit",
            "data_name=hirise_net",
            "seed=0",
            "training.early_stopping_patience=5",
        ],
    )
    cfg = load_dynamic_configs(cfg)

In [3]:
import pandas as pd
pd.read_csv(cfg.data.annot_csv)

,image_path,label,split
0,ESP_013049_0950_RED-0067.jpg,7,train
1,ESP_013049_0950_RED-0067-fv.jpg,7,train
2,ESP_013049_0950_RED-0067-brt.jpg,7,train
3,ESP_013049_0950_RED-0067-r90.jpg,7,train
4,ESP_013049_0950_RED-0067-r180.jpg,7,train
...,...,...,...
67805,ESP_018707_2205_RED-0041.jpg,0,test
67806,ESP_018707_2205_RED-0062.jpg,0,test
67807,ESP_018707_2205_RED-0105.jpg,0,test
67808,ESP_018707_2205_RED-0058.jpg,0,test


In [2]:
from omegaconf import OmegaConf
print(OmegaConf.to_yaml(cfg))

task: classification
mode: train
data_name: hirise_net
model_name: vit
dataset_path: /data/hkerner/MarsBench/Datasets
output_path: outputs
seed: 0
test_after_training: false
prediction_output_path: null
checkpoint_path: null
data:
  split:
    train: 0.6
    val: 0.2
    test: 0.2
  valid_image_extensions:
  - jpg
  - JPG
  - jpeg
  - JPEG
  - png
  - PNG
  - tif
  - TIF
  name: HiRISENet
  status: test
  data_dir: ${oc.decode:${oc.select:dataset_path, .}}/Mars_Image_Cont_Class_Landmark/hirise-map-proj-v3_2/map-proj-v3_2/
  annot_csv: ${oc.decode:${oc.select:dataset_path, .}}/Mars_Image_Cont_Class_Landmark/hirise-map-proj-v3_2/annotation.csv
  num_classes: 8
  image_type: rgb
  txt_file: ${oc.decode:${oc.select:dataset_path, .}}/Mars_Image_Cont_Class_Landmark/hirise-map-proj-v3_2/labels-map-proj_v3_2_train_val_test.txt
transforms:
  image_size:
  - 224
  - 224
  rgb:
    mean:
    - 0.485
    - 0.456
    - 0.406
    std:
    - 0.229
    - 0.224
    - 0.225
  grayscale:
    mean:
    - 

# Prepare Data

In [3]:
import sys
sys.path.append('..')
from marsbench.data import *
from marsbench.utils.transforms import get_transforms

In [4]:
# Just for explaination purpose, doesn't need to be run
train_transform, val_transform = get_transforms(cfg)
train_dataset, val_dataset, test_dataset = get_dataset(cfg, [train_transform, val_transform])
# Just for explaination purpose, doesn't need to be run
train_transform, val_transform = get_transforms(cfg)
train_dataset, val_dataset, test_dataset = get_dataset(cfg, [train_transform, val_transform])

In [5]:
from marsbench.data.mars_datamodule import MarsDataModule
data_module = MarsDataModule(cfg)
data_module.setup()

train_loader = data_module.train_dataloader()
val_loader = data_module.val_dataloader()
test_loader = data_module.test_dataloader()

/home/vmalaviy/.conda/envs/vl/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Load Model

### Config

In [6]:
import os
from datetime import datetime
from pytorch_lightning import Trainer
from marsbench.models import import_model_class
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

timestamp = datetime.now().strftime(r"%m%d_%H%M")
job_name = 'trial'

output_path = f'outputs/{cfg.task}/{job_name}/{timestamp}'

os.makedirs(output_path, exist_ok=True)
os.makedirs(f'{output_path}/checkpoints', exist_ok=True)
os.makedirs(f'{output_path}/wandb', exist_ok=True)

### Callbacks

In [7]:
wandb_logger = WandbLogger(
    project="MarsBench",
    name=job_name,
    log_model=True,
    save_dir=output_path,
    dir=f'{output_path}/wandb'
)

callbacks = [
    ModelCheckpoint(
        dirpath=f'{output_path}/checkpoints',
        filename="{epoch}",
        save_top_k=1,
        monitor="val/acc",
        mode="max",
        verbose=True
    ),
    EarlyStopping(
        monitor="val/acc",
        mode="max",
        patience=cfg.training.early_stopping_patience,
    ),
]

### init

In [8]:
model_class = import_model_class(cfg)
model = model_class(cfg)

In [9]:
# Fast Check
# trainer = Trainer(max_epochs=cfg.training.max_epochs, fast_dev_run=True)

# Mini Training
trainer_config = {
    k: v for k, v in cfg.training.trainer.items() if k not in ["logger"]
}

trainer = Trainer(
    max_epochs= 2, #cfg.training.trainer.max_epochs,
    logger=wandb_logger,
    callbacks=callbacks,
    enable_model_summary=True,
    enable_progress_bar=True,
    limit_train_batches=0.3,
    limit_val_batches=0.3,
    limit_test_batches=0.3,
    accumulate_grad_batches=2,
    gradient_clip_val=0.5
)

/home/vmalaviy/.conda/envs/vl/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/vmalaviy/.conda/envs/vl/lib/python3.11/site-pa ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# Training

In [10]:
model.train()
trainer.fit(
    model, train_dataloaders=train_loader, val_dataloaders=val_loader
)

You are using a CUDA device ('NVIDIA A30') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vmalaviy (vmalaviy-arizona-state-university). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | VisionTransformer | 303 M  | train
1 | criterion | CrossEntropyLoss  | 0      | train
--------------------------------------------------------
8.2 K     Trainable params
303 M     Non-trainable params
303 M     Total params
1,213.239 Total estimated model params size (MB)
297       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/vmalaviy/.conda/envs/vl/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 239: 'val/acc' reached 0.82054 (best 0.82054), saving model to '/home/vmalaviy/MarsBench/outputs/classification/trial/0321_0011/checkpoints/epoch=0.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 1, global step 478: 'val/acc' reached 0.84241 (best 0.84241), saving model to '/home/vmalaviy/MarsBench/outputs/classification/trial/0321_0011/checkpoints/epoch=1.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=2` reached.


# Test

In [11]:
model.eval()
res = trainer.test(
    model, dataloaders=test_loader
)
res

/home/vmalaviy/.conda/envs/vl/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/vmalaviy/.conda/envs/vl/lib/python3.11/site-pa ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/acc          │    0.8639705777168274     │
│         test/loss         │    0.5565390586853027     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.5565390586853027, 'test/acc': 0.8639705777168274}]

In [12]:
out = trainer.predict(model, test_loader)
len(out)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

57